In [1]:
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
calendar = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/calendar.csv')
sales_train_val = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/sales_train_validation.csv')
sell_prices = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/sell_prices.csv')
#sales_train_evaluation = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/sales_train_evaluation.csv')

모르는 변수 정리
- wm_yr_wk :The id of the week the date belongs to
- wday : 요일을 숫자로 바꾼 것(1: 토, 2:일 ..)

---

## calendar

In [3]:
calendar.head(10)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0


In [4]:
print('calendar shape:',calendar.shape)
print('마지막 날짜',calendar.loc[1968,'d'])

calendar shape: (1969, 14)
마지막 날짜 d_1969


calendar 데이터에는 Train_set의 첫번째 날짜 (d_1)부터 최종 Test_set의 마지막 날짜(d_1969)까지 모두 정의되어 있음

In [5]:
event_type_list=list(set(list(calendar['event_type_1'].unique()) + list(calendar['event_type_2'].unique())))
print(event_type_list)
event_type_list = event_type_list[1:]
print(event_type_list)

[nan, 'National', 'Cultural', 'Sporting', 'Religious']
['National', 'Cultural', 'Sporting', 'Religious']


In [6]:
event_name_list= list(set(list(calendar['event_name_1'].unique()) + list(calendar['event_name_2'].unique())))
print(event_name_list)
event_name_list = event_name_list[1:]
print(event_name_list)

[nan, 'LaborDay', 'Cinco De Mayo', 'SuperBowl', 'Eid al-Fitr', "Mother's day", "Father's day", 'Chanukah End', 'NewYear', 'NBAFinalsStart', 'StPatricksDay', 'Thanksgiving', 'OrthodoxChristmas', 'Easter', 'LentWeek2', 'Christmas', 'MartinLutherKingDay', 'PresidentsDay', 'Halloween', 'MemorialDay', 'Ramadan starts', 'EidAlAdha', 'LentStart', 'VeteransDay', 'Purim End', 'ValentinesDay', 'OrthodoxEaster', 'NBAFinalsEnd', 'ColumbusDay', 'Pesach End', 'IndependenceDay']
['LaborDay', 'Cinco De Mayo', 'SuperBowl', 'Eid al-Fitr', "Mother's day", "Father's day", 'Chanukah End', 'NewYear', 'NBAFinalsStart', 'StPatricksDay', 'Thanksgiving', 'OrthodoxChristmas', 'Easter', 'LentWeek2', 'Christmas', 'MartinLutherKingDay', 'PresidentsDay', 'Halloween', 'MemorialDay', 'Ramadan starts', 'EidAlAdha', 'LentStart', 'VeteransDay', 'Purim End', 'ValentinesDay', 'OrthodoxEaster', 'NBAFinalsEnd', 'ColumbusDay', 'Pesach End', 'IndependenceDay']


In [7]:
calendar[calendar['event_type_1'] == calendar['event_type_2']]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI


서로 type이 겹치는 날은 없는 것으로 보임.

---

## sell_prices

In [8]:
sell_prices.shape

(6841121, 4)

In [9]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [10]:
print(sell_prices['wm_yr_wk'].value_counts())

11618    30490
11614    30490
11619    30490
11605    30490
11604    30490
         ...  
11105    13033
11104    12799
11103    12519
11102    11975
11101    10932
Name: wm_yr_wk, Length: 282, dtype: int64


sell_prices의 'wm_yr_wk' 변수의 개수가 다른 것으로 보아 각각의 주(week)마다 총 판매되는 상품의 개수가 다른 것으로 확인 됨.

In [0]:
sell_prices['id' ]= sell_prices['item_id']+'_'+sell_prices['store_id']
sell_prices.drop(['store_id','item_id'],axis=1,inplace=True)

---

## sales_train_val
- 최종적으로 이 데이터로 시계열 프레임을 만든 다음에 예측을 해야함.

In [12]:
sales_train_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [13]:
sales_train_val['store_id'].unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

In [14]:
sales_train_val['item_id'].unique().shape

(3049,)

3049개의 품목 10개의 상점
- 3049 X 10 = 30490

In [15]:
sales_train_val['dept_id'].unique()

array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
       'FOODS_2', 'FOODS_3'], dtype=object)

# EDA

## calendar, sell_prcies merge
- Ram 과부화로 안하는 것이 좋을 것 같음.

In [0]:
# merge_1=sell_prices.merge(calendar,on='wm_yr_wk')
# merge_1.head()

In [0]:
# merge_1.shape

In [0]:
# merge_1['days']=merge_1['d'].apply(lambda x: int(x[2:]))

In [0]:
# merge_1.sort_values('days',inplace=True)

In [0]:
# merge_1['id'] = merge_1['item_id']+ '_' + merge_1['store_id']
# merge_1.drop(['store_id','item_id'],axis=1,inplace=True)
# merge_1.head()

In [0]:
# del sell_prices
# del calendar
# gc.collect()

In [0]:
# merge_1.shape[0]

In [0]:
# print(sales_train_val.drop(['id','item_id','dept_id','cat_id','store_id','state_id'],axis=1).shape)
# 30490*1913

sales_train_val 데이터와 merge_1 데이터와 개수가 맞지 않다. 특정날짜(특정 주 week)에 아예 존재하지 않는 sale_prices가 있는 것 같다.
- train 데이터를 변형시켜서 결합시켜야 할 것 같음.
- 하지만 merge데이터를 변형된 train에 붙일 수는 있을 것 같음.

---

## To  change sales_train_val for modeling

In [24]:
sales_data=sales_train_val.iloc[:,6:]
sales_data['id'] = sales_train_val['id']

del sales_train_val
gc.collect()

sales_data = sales_data.melt(id_vars='id')
sales_data.id.replace(regex='_validation',value='',inplace=True)
sales_data.rename({'variable':'d'},axis=1,inplace=True)

sales_data['d'].replace(regex ='d_',value='',inplace=True)
sales_data['d'] = sales_data['d'].astype('int16')
calendar['d'].replace(regex ='d_',value='',inplace=True)
calendar['d'] = calendar['d'].astype('int16')

sales_data = sales_data.merge(calendar[['d','wm_yr_wk']],on='d')

sales_data = sales_data.merge(sell_prices[['id','wm_yr_wk','sell_price']],on=['id','wm_yr_wk'],how='left')

# sales_data['state'] = sales_data.id.apply(lambda x: x[-4:-2])
# sales_data['market'] = sales_data.id.apply(lambda x: x[-4:])

sales_data.head()

,id,d,value,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1,1,0,11101,NaN
1,HOBBIES_1_002_CA_1,1,0,11101,NaN
2,HOBBIES_1_003_CA_1,1,0,11101,NaN
3,HOBBIES_1_004_CA_1,1,0,11101,NaN
4,HOBBIES_1_005_CA_1,1,0,11101,NaN


calendar 데이터의 evenet_name과 event_type 데이터를 원핫 인코딩 형식으로 sales_data에 삽입해보자.

In [0]:
def make_onehot_type_col(col):
  sales_data[col] = 0
  for day in calendar.loc[(calendar['event_type_1'] == col)|(calendar['event_type_2'] == col),'d'].values:
    sales_data.loc[sales_data['d']==day, col] +=1

def make_onehot_name_col(col):
  sales_data[col] = 0
  for day in calendar.loc[(calendar['event_name_1'] == col)|(calendar['event_name_2'] == col),'d'].values:
    sales_data.loc[sales_data['d']==day, col] +=1

In [0]:
for col in event_type_list:
  make_onehot_type_col(col)

In [0]:
sales_data.to_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/sales_data.csv')

In [29]:
sales_data.head()

,id,d,value,wm_yr_wk,sell_price,National,Cultural,Sporting,Religious
0,HOBBIES_1_001_CA_1,1,0,11101,NaN,0,0,0,0
1,HOBBIES_1_002_CA_1,1,0,11101,NaN,0,0,0,0
2,HOBBIES_1_003_CA_1,1,0,11101,NaN,0,0,0,0
3,HOBBIES_1_004_CA_1,1,0,11101,NaN,0,0,0,0
4,HOBBIES_1_005_CA_1,1,0,11101,NaN,0,0,0,0


In [0]:
for col in event_name_list:
  make_onehot_name_col(col)

event_name을 더미화시키는데 RAM 과부화가 발생한다. 지역별로 쪼개서 csv를 다뤄야 할 것 같음.